In [3]:
#BASIC PATH TO YOLO-DIR
MODEL_PATH = "yolo-coco"


#THE MINIMUM CONFIDENCE OF OBJ
MIN_CONFID = 0.3


#MINIMUM THRESHOLD FOR NONMAXIMA SUPPRESSION
NMS_THRESHLD = 0.3


#WHETHER TO USE GPU OR NOT
USE_GPU = False


#MINIMUM SOCIAL-DISTANCE
MIN_DISTANCE = 50

In [4]:
import numpy as np
import cv2

#makeing frames from social_detection
def detect_people(frame, net, ln, personIdx=0):
    #grabbing the dimensions of the frame and initializing the list results
    #net: The pre-trained and pre-initialized YOLO object detection model
    #ln: YOLO CNN output layer names

    (H, W) = frame.shape[:2]
    results = []
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    
    #net.setInput(blob) - passing the images to the algorithm or you can say into the network
    #net.forward(ln) – It will return us the bounded boxes coordinate, centroids and confidence values from the image. 

    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    boxes = []
    centroids = []
    confidences = []

    for output in layerOutputs:
        for detection in output:
            #confidence, x, y, h, w, pr(1), pr(2), pr(3)
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if classID == personIdx and confidence > MIN_CONFID:
                
                #scale the bounding box coordinates back relative to the size of the image
                
                centerX=int(detection[0]*W)
                centerY=int(detection[1]*H)
                width=int(detection[2]*W)
                height=int(detection[3]*H)
                
                #box = detection[0:4] * np.array([W, H, W, H])
                #(centerX, centerY, width, height) = box.astype("int")

                #use the center (x, y)-coordinates to derive the top and left corner of the bounding box

                x = int(centerX - (width/2))
                y = int(centerY - (height/2))
                
                
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))
    
    #Non Maxima Supression
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONFID, NMS_THRESHLD)
    
    if len(idxs) > 0:
        #loop over the indexes we are keeping
        for i in idxs.flatten():
            #extracting bounding box coordinates
            
            x, y, w, h = boxes[i]
            
            #(x, y) = (boxes[i][0], boxes[i][1])
            #(w, h) = (boxes[i][2], boxes[i][3])
            
            #updating our Results list = consisting persons
            #predicting probability, Bounding Box coordinates, centriod
            
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)
        
#return the list of results
    return results
            
    
# WORKING....
#frames get preprocessed and are given back to the model and gets o/p fromit
#compared based only on people/persons and returned
#NMS - NON MAXIMA SUPRESSION
#gives back CENTRIOD, BOUNDING BOX CORDINATES, CONFIDENCE

In [7]:
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2
import os

labelsPath =os.path.sep.join([MODEL_PATH,"coco.names"])
LABELS =open(labelsPath).read().strip().split("\n")

weightsPath =os.path.sep.join([MODEL_PATH,"yolov3.weights"])

configPath= os.path.sep.join([MODEL_PATH,"yolov3.cfg"])
print("[INFO] loading YOLO from disk")

#net: The pre-trained and pre-initialized YOLO object detection model
    
net =cv2.dnn.readNetFromDarknet(configPath,weightsPath)

if USE_GPU:
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


#ln: YOLO CNN output layer names

ln =net.getLayerNames()
ln = [ln[i[0]-1]for i in net.getUnconnectedOutLayers()]
print("[INFO] accesing video stream")

vs =cv2.VideoCapture( r"pedestrian.mp4"if "pedestrian.mp4"else 0)#r"pedestrian.mp4"if "pedestrian.mp4"else
writer =None

while True:
    (grabbed,frame) =vs.read()
    if not grabbed:
        break
    frame = imutils.resize(frame,width =700)
    results =detect_people(frame,net,ln,personIdx=LABELS.index("person"))
    violate =set()
    
    if len(results) >=2:
        
        # extract all centroids from the results and compute the Euclidean distances between all pairs of the centroids
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids,centroids,metric="euclidean")
        
        
   # loop over the upper triangular of the distance matrix
   
        for i in range(0,D.shape[0]):
            for j in range(i+1,D.shape[1]):
                if D[i,j]<MIN_DISTANCE:
                    violate.add(i)
                    violate.add(j)
                    
                    
    for(i,(prob,bbox,centroid)) in enumerate(results):
        (startX,startY,endX,endY)=bbox
        (cX,cY)=centroid 
        color=(0,255,0)
        
        if i in violate:
            color = (0,0,255)
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
        cv2.circle(frame,(cX,cY),5,color,1)
    
    text ="Social Distancing Violations: {}".format(len(violate))
    cv2.putText(frame,text,(10,frame.shape[0]-25),cv2.FONT_HERSHEY_SIMPLEX,0.85,(0,0,255),3)
    cv2.imshow("Frame",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

    if r"social-distance-detector" != "" and writer is None:
        fourcc =cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(r"output.mp4",fourcc,25,(frame.shape[1],frame.shape[0]),True)
        
    if writer is not None:
        writer.write(frame)
        cv2.destroyAllWindows()

[INFO] loading YOLO from disk
[INFO] accesing video stream


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-ubn7l1_7\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
